In [1]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [4]:
#independencies
import requests
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import branca.colormap as cm
import folium
import datetime
import time
from folium.plugins import TimeSliderChoropleth
from ipywidgets import widgets
from ipywidgets import interact, interactive
from IPython.display import display, IFrame, HTML

In [5]:
#get datas from JHU
covid19 = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
#print(covid19.columns)
#extract data we want
col_name = ["iso_code","location","date","total_cases","new_cases","total_deaths","new_deaths","new_cases_per_million","new_deaths_per_million"]
covid19 = covid19[col_name]
#define countries boundries using gpd
country_boundries = gpd.read_file('country_boundries/country_boundries.shp')
country_boundries = country_boundries[["CNTRY_NAME","geometry"]]

In [6]:
#convert data types
covid19.fillna(0, inplace=True)
covid19.replace(np.nan, 0, inplace=True)
covid19.replace(np.inf, 0, inplace=True)
for col in covid19.columns[3:8]: 
    covid19[col]=covid19[col].astype(int)

In [7]:
#merge dataframe of country boundries into covid's dataframe
country_boundries = country_boundries.rename(columns = {"CNTRY_NAME" : "location"})
Covid19 = pd.merge(covid19, country_boundries, how='left', on='location')

In [8]:
Covid19.head(1)

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,new_cases_per_million,new_deaths_per_million,geometry
0,AFG,Afghanistan,2020-02-24,1,1,0,0,0,0.0,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."


In [9]:
#convert number of cases and deaths in logarithm
Covid19['total_cases_log'] = np.log10(Covid19['total_cases'] + 1)
Covid19['new_cases_log'] = np.log10(Covid19['new_cases'] + 1)
Covid19['total_deaths_log'] = np.log10(Covid19['total_deaths'] + 1)
Covid19['new_deaths_log'] = np.log10(Covid19['new_deaths'] + 1)
Covid19['new_cases_per_million_log'] = np.log10(Covid19['new_cases_per_million'] + 1)
Covid19['new_deaths_per_million_log'] = np.log10(Covid19['new_deaths_per_million'] + 1)

#convert date in timestamped format
Covid19['date_sec_int'] = pd.to_datetime(Covid19['date']).astype(int) / 10**9
Covid19['date_sec'] = Covid19['date_sec_int'].astype(int).astype(str)

/Users/emiliekong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/emiliekong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/emiliekong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log10
  """
/Users/emiliekong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log10
  """
/Users/emiliekong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log10
  
/Users/emiliekong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log10
  
/Users/emiliekong/anaco

In [10]:
#preparation of dataframe to generate graph
log_confirmed = Covid19[['location', 'date_sec', 'total_cases_log', 'geometry']]
#clear NaN
log_confirmed = log_confirmed.dropna(axis=0,how='any')
#define map color using number of confirmed cases
max_colour = max(log_confirmed['total_cases_log'])
min_colour = min(log_confirmed['total_cases_log'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
log_confirmed['colour'] = log_confirmed['total_cases_log'].map(cmap)

In [11]:
#we construct our style dictionnary
country_list = log_confirmed['location'].unique().tolist()
country_idx = range(len(country_list))

style_dict = {}
for i in country_idx:
    country = country_list[i]
    result = log_confirmed[log_confirmed['location'] == country]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

In [12]:
#dataframe containing features for each country
countries_df = log_confirmed[['geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
#IMPORTANT,if not,the program will be running forever
countries_gdf = countries_gdf.drop_duplicates().reset_index()

In [13]:
#create our map with a slidebar
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service
slider_tt_cases = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')

TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_tt_cases)

cmap.add_to(slider_tt_cases)
cmap.caption = "Log of number of confirmed cases"
slider_tt_cases.save(outfile='slider_tt_cases.html')

In [14]:
#preparation of dataframe to generate graph
log_deaths = Covid19[['location', 'date_sec', 'total_deaths_log', 'geometry']]
#clear NaN
log_deaths = log_deaths.dropna(axis=0,how='any')
#define map color using number of deaths
max_colour = max(log_deaths['total_deaths_log'])
min_colour = min(log_deaths['total_deaths_log'])
cmap2 = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
log_deaths['colour'] = log_deaths['total_deaths_log'].map(cmap2)

style_dict_deaths = {}
for i in country_idx:
    country = country_list[i]
    result = log_deaths[log_deaths['location'] == country]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict_deaths[str(i)] = inner_dict

slider_tt_deaths = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')

TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict_deaths,

).add_to(slider_tt_deaths)

cmap2.add_to(slider_tt_deaths)
cmap2.caption = "Log of number of deaths in total"
slider_tt_deaths.save(outfile='slider_tt_deaths.html')

In [15]:
#every day's new cases

#preparation of dataframe to generate graph
log_cases_today = Covid19[['location', 'date_sec', 'new_cases_log', 'geometry']]
#clear NaN and inf
log_cases_today = log_cases_today.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
#define map color using number of deaths
max_colour = max(log_cases_today['new_cases_log'])
min_colour = min(log_cases_today['new_cases_log'])
min_colour = 0
cmap3 = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
log_cases_today['colour'] = log_cases_today['new_cases_log'].map(cmap3)

style_dict_new_cases = {}
for i in country_idx:
    country = country_list[i]
    result = log_cases_today[log_cases_today['location'] == country]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict_new_cases[str(i)] = inner_dict

slider_new_cases = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')

TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict_new_cases,

).add_to(slider_new_cases)

cmap3.add_to(slider_new_cases)
cmap3.caption = "Log of number of each day's new cases"
slider_new_cases.save(outfile='slider_new_cases.html')

In [16]:
#every day's new deaths

#preparation of dataframe to generate graph
deaths_today = Covid19[['location', 'date_sec', 'new_deaths',"new_deaths_log" ,'geometry']]
#clear NaN and inf
deaths_today = deaths_today.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
#define map color using number of deaths
max_colour = max(deaths_today['new_deaths_log'])
min_colour = min(deaths_today['new_deaths_log'])
cmap4 = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
deaths_today['colour'] = deaths_today['new_deaths_log'].map(cmap4)

style_dict_new_deaths = {}
for i in country_idx:
    country = country_list[i]
    result = deaths_today[deaths_today['location'] == country]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict_new_deaths[str(i)] = inner_dict

slider_new_deaths = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')

TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict_new_deaths,
).add_to(slider_new_deaths)

cmap4.add_to(slider_new_deaths)
cmap4.caption = "Log of number of each day's new deaths"
slider_new_deaths.save(outfile='slider_new_deaths.html')

In [17]:
#today's data
today = Covid19.date_sec_int.unique().max()
today_df = Covid19[Covid19.date_sec_int >= today - int(86400)]#last 24 hours
today_df = today_df.dropna()
today_df = today_df.rename(columns = {"location" : "Country"})

In [18]:
#today's total cases' map

#base map

map_tt_cases = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')
today_tt_cases = today_df[["Country","total_cases_log","total_cases","geometry"]]
today_tt_cases = today_tt_cases.rename(columns = {"total_cases" : "Total cases"})
today_tt_cases = today_tt_cases.rename(columns = {"total_cases_log" : "total_cases"})

#create a dictionary from country names to total cases
df = today_tt_cases[['Country', 'total_cases']]
df = df.drop_duplicates(subset=['Country'], keep='first') 
df.set_index(keys='Country', inplace=True)
df = df.T

#create a geojson file for tooltip
crs = {'init': 'epsg:4326'}
tt_cases_gpd = gpd.GeoDataFrame(today_tt_cases, crs=crs, geometry ='geometry')
gjson = tt_cases_gpd.to_crs(epsg='4326').to_json()

colorscale = cm.linear.YlOrRd_09.scale(today_tt_cases.total_cases.min(), today_tt_cases.total_cases.max())


def style_function(feature):
    total_cases = int(df.get(feature['properties']["Country"]))
    return {
        'fillOpacity': 0.7,
        'weight': 1,
        "color" : 'grey', 
        "dashArray": "5, 5",
        'fillColor': 'black' if total_cases is None else colorscale(total_cases)
    }
colorscale.add_to(map_tt_cases)
colorscale.caption = 'Log of total cases by Country'
country = folium.features.GeoJson(gjson,                                  
                                  tooltip=folium.features.GeoJsonTooltip(fields=["Country",'Total cases']),
                                  style_function=style_function)
map_tt_cases.add_child(country)
map_tt_cases.save(outfile='_tt_cases.html')

In [19]:
#today's total deaths' map

#base map

map_tt_deaths = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')
today_tt_deaths = today_df[["Country","total_deaths_log","total_deaths","geometry"]]
today_tt_deaths = today_tt_deaths.rename(columns = {"total_deaths" : "Total Deaths"})
today_tt_deaths = today_tt_deaths.rename(columns = {"total_deaths_log" : "total_deaths"})

#create a dictionary from country names to total cases
df2 = today_tt_deaths[['Country', 'total_deaths']]
df2 = df2.drop_duplicates(subset=['Country'], keep='first') 
df2.set_index(keys='Country', inplace=True)
df2 = df2.T

#create a geojson file for tooltip
crs = {'init': 'epsg:4326'}
tt_deaths_gpd = gpd.GeoDataFrame(today_tt_deaths, crs=crs, geometry ='geometry')
gjson_tt_deaths = tt_deaths_gpd.to_crs(epsg='4326').to_json()

colorscale2 = cm.linear.YlOrRd_09.scale(today_tt_deaths.total_deaths.min(), today_tt_deaths.total_deaths.max())


def style_function(feature):
    total_deaths = int(df2.get(feature['properties']["Country"]))
    return {
        'fillOpacity': 0.7,
        'weight': 1,
        "color" : 'grey', 
        "dashArray": "5, 5",
        'fillColor': 'black' if total_deaths is None else colorscale2(total_deaths)
    }
colorscale2.add_to(map_tt_deaths)
colorscale2.caption = 'Log of total deaths by Country'
tt_deaths = folium.features.GeoJson(gjson_tt_deaths,                                  
                                  tooltip=folium.features.GeoJsonTooltip(fields=["Country",'Total Deaths']),
                                  style_function=style_function)
map_tt_deaths.add_child(tt_deaths)
map_tt_deaths.save(outfile='_tt_deaths.html')

In [20]:
#today's new cases' map

#base map

map_new_cases = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')
new_cases = today_df[["Country","new_cases","new_cases_log","geometry"]]

#create a dictionary from country names to total cases
df3 = new_cases[['Country', 'new_cases_log']]
df3 = df3.drop_duplicates(subset=['Country'], keep='first') 
df3.set_index(keys='Country', inplace=True)
df3 = df3.T

#create a geojson file for tooltip
crs = {'init': 'epsg:4326'}
new_cases_gpd = gpd.GeoDataFrame(new_cases, crs=crs, geometry ='geometry')
gjson_new_cases = new_cases_gpd.to_crs(epsg='4326').to_json()

colorscale3 = cm.linear.YlOrRd_09.scale(new_cases.new_cases_log.min(), new_cases.new_cases_log.max())


def style_function(feature):
    new_cases_log = int(df3.get(feature['properties']["Country"]))
    return {
        'fillOpacity': 0.7,
        "color" : 'grey', 
        'weight': 1,
        "dashArray": "5, 5",
        'fillColor': 'black' if new_cases_log is None else colorscale3(new_cases_log)
    }
colorscale3.add_to(map_new_cases)
colorscale3.caption = 'Log of new cases in last 24 hours by Country'
today_cases = folium.features.GeoJson(gjson_new_cases,                                  
                                  tooltip=folium.features.GeoJsonTooltip(fields=["Country",'new_cases']),
                                  style_function=style_function)
map_new_cases.add_child(today_cases)
map_new_cases.save(outfile='_new_cases.html')

In [21]:
#today's new deaths' map

#base map

map_new_deaths = folium.Map(min_zoom=1.6, max_bounds=True,height=700,width=1200, tiles=tileset,attr='Yuexuan KONG')
new_deaths = today_df[["Country","new_deaths","geometry"]]

#create a dictionary from country names to total cases
df4 = new_deaths[['Country', 'new_deaths']]
df4 = df4.drop_duplicates(subset=['Country'], keep='first') 
df4.set_index(keys='Country', inplace=True)
df4 = df4.T

#create a geojson file for tooltip
crs = {'init': 'epsg:4326'}
new_deaths_gpd = gpd.GeoDataFrame(new_deaths, crs=crs, geometry ='geometry')
gjson_new_deaths = new_deaths_gpd.to_crs(epsg='4326').to_json()

colorscale4 = cm.linear.YlOrRd_09.scale(new_deaths.new_deaths.min(), new_deaths.new_deaths.max())
colorscale4.caption = 'New deaths in last 24 hours by Country'


def style_function(feature):
    new_deaths = int(df4.get(feature['properties']["Country"]))
    return {
        'fillOpacity': 0.7,
        'weight': 1,
        "color" : 'grey', 
        "dashArray": "5, 5",
        'fillColor': 'black' if new_deaths is None else colorscale4(new_deaths)
    }
colorscale4.add_to(map_new_deaths)
colorscale4.caption = 'New deaths in last 24 hours by Country'
today_deaths = folium.features.GeoJson(gjson_new_deaths,                                  
                                  tooltip=folium.features.GeoJsonTooltip(fields=["Country",'new_deaths']),
                                  style_function=style_function)
map_new_deaths.add_child(today_deaths)
map_new_deaths.save(outfile='_new_deaths.html')